
<div align="right" style="text-align:right"><i>Peter Norvig<br>May 2020</i></div>

# Flipping Cards: A Guessing Game

The [538 Riddler for Aug. 17, 2018](
https://fivethirtyeight.com/features/step-1-game-theory-step-2-step-3-profit/) poses this problem:

> Take a standard deck of cards, and pull out the numbered cards from one suit (the nine cards 2 through 10). Shuffle them, and then lay them face down in a row. Flip over the first card so it is face up. Now guess whether the next card in the row (the first down card) is higher or lower. If you’re right, keep going. If you play this game optimally, what’s the probability that you can get to the end without making any mistakes?<p>
> *Extra credit:* What if there were more cards — 2 through 20, or 2 through 100? How do your chances of getting to the end change?
    
First, let's define "*play this game optimally.*"  The optimal policy is to look at the current flipped-up card and, remembering what cards have already been flipped and thus knowing what cards remain face-down, count how many of the remaining down cards are higher than the up card, and how many are lower. Guess according to the majority. (*Details*: In case of a tie in the counts, guessing either "higher" or "lower" is equally optimal. When there are only two cards left (one up, one down) you know for sure what the down card is, so you can always guess right.)
    
We can solve the problem with brute force: define `guess_all(cards)` to take a specific sequence of cards and return true only if the optimal policy guesses all the cards correctly. Then we can apply `guess_all` to every permutation of cards and count how often it guesses correctly. Assuming every permutation is equally likely, the ratio is the probability of guessing right all the way to the end.
    
(*Python Trivia*: `True` is equal to `1` and `False` is equal to `0`, so `sum([True, False, True])` is 2; `mean([True, False, True])` is 2/3, and the expression `(b and x)` where `b` is a Boolean and `x` is a number is equivalent to `(x if b else 0)`. In other languages you would be required or at least encouraged to explicitly cast Booleans to integers, but in Python it is considered good style to do without explicit casts.)
    
I can implement `guess_all` as follows:

In [1]:
from itertools  import permutations
from statistics import mean
from functools  import lru_cache

In [2]:
def guess_all(cards) -> bool: 
    """You guess all the cards right if you guess the first right and the rest right."""
    return len(cards) <= 2 or guess_first(cards) and guess_all(cards[1:])

def guess_first(cards) -> bool:
    """Given a sequence of cards, guess that the down card will be "higher" or "lower" 
    according to majority counts of the down cards; return True if the guess 
    matches the actual relation between the up card and the first down card."""
    up, *down = cards
    higher = sum(d > up for d in down)
    lower  = sum(d < up for d in down)
    return (higher > lower) == (down[0] > up)

Some simple examples:

In [3]:
 # On this sequence, the optimal policy always guesses "higher"; that's right.  
guess_all((2, 3, 4, 5, 6))

True

In [4]:
# On this sequence, the optimal policy guesses "higher" first; that's wrong.
# There is a 3/4 chance that a down card is higher than 3, but it turns
# out that the 2 is actually lower.
guess_all((3, 2, 4, 5, 6)) 

False

In [5]:
guess_first((3, 2, 4, 5, 6))

False

# The Answer

What's the probability that we can guess all nine cards right? We get that by averaging over all possible permutations of the cards:

In [6]:
cards = range(2, 11)

mean(map(guess_all, permutations(cards)))

0.17085537918871252

That's the correct answer ([per 538](https://fivethirtyeight.com/features/how-many-hoops-will-kids-jump-through-to-play-rock-paper-scissors/)): about 17% chance of guessing all nine cards right.

# Extra Credit

The extra credit problem asks about 19 and 99 cards. But there are 19! permutations of 19 cards; that's over $10^{17}$ (and don't even think about 99!). So my brute force solution won't work. We will need a more efficient way of looking at the space of possible sequences of cards. 

I note that two things are true when we are deciding whether to guess "higher" or "lower" in `guess_first(cards)`: 

**First**, the **order** of the `down` cards doesn't matter. The optimal policy makes the same guess whether `down` is `(2, 3, 4, 5)` or a permutation such as `(4, 2, 5, 3)`. So we could represent cards as a set: `{2, 3, 4, 5}`. As we flip over cards the set of remaining cards becomes smaller; altogether there are $2^n$ subsets to consider, a significant reduction from the $n!$ permutations we had to consider in the brute force approach, and good enough to handle $n=19$ (but not yet good enough for $n=99$).

**Second**, the **identity** of the `down` cards doesn't matter (if we're careful).
For example, when the up card is `3` it doesn't matter if the remaining cards are `{2, 3, 6}` or `{1, 3, 4}` or `{0, 3, 5}` or any other set of three cards in which the `3` is the middle card. For any such set, the chance of guessing right is 50%. Note that I can't just summarize these situations as "*there are 3 cards remaining and the up card is 3*" but I can summarize them as  "*there are 3 cards remaining and the up card is the second lowest*". In other words, I describe the up card by **renumbering** it to be its index into the list of sorted cards. For each of the three sets described here, the index of `3` is `1`: it is the second element of the sorted list of cards. Some examples:

  - [Remaining: `{2, 3, 6}` Up card: `3`] with renumbering becomes [Remaining: `range(3)` Up index: `1`] (i.e., second lowest)
  - [Remaining: `{1, 3, 4}` Up card: `3`] with renumbering becomes [Remaining: `range(3)` Up index: `1`] (i.e., second lowest)
  - [Remaining: `{0, 3, 5}` Up card: `3`] with renumbering becomes [Remaining: `range(3)` Up index: `1`] (i.e., second lowest)
  - [Remaining: `{3, 4, 7}` Up card: `3`] with renumbering becomes [Remaining: `range(3)` Up index: `0`] (i.e., lowest)
  - [Remaining: `{0, 2, 3}` Up card: `3`] with renumbering becomes [Remaining: `range(3)` Up index: `2`] (i.e., third lowest)

  
Renumbering the up card as an index number won't alter the probabilities, but it will allow me to describe any situation with just two integers, the number of cards remaining and the index of the up card. That means there are only $O(n^2)$ situations to describe, which should be good enough to handle $n=99$. 

In other words, what I have done is gone from a **concrete** representation of the cards as explicit permutations to an **abstract** representation that says how many cards remain and what the index number of the up card is, but doesn't say exactly what the cards are nor what order they are in. Abstraction is often a good way to fight against combinatorial explosion.




# First Try at Extra Credit

I thought I could define a function, `P_all(n)`, that told me the probability of guessing all `n` cards right. I wrote down a first try at a definition:

    def P_all(n) -> float:
        return 1 if n <= 2 else P_first(n) * P_all(n - 1)
        
Could it really be as simple as that? All I have to do is define `P_first(n)` (the probability of guessing the first of `n` cards right) and I'm done? My immediate reaction was that this feels *too* easy. I was suspicious of this definition for three reasons:

1. It is rare (but not unheard of) for an efficient solution to be simpler than the brute force solution.
- I said there should be renumbering, but this code doesn't appear to do any renumbering.
- Yes, the probability of getting all the cards right can be described as the probability of getting the first card right and then getting the rest right. But the joint probability of two events is equal to the product of their probabilities *only when the two events are independent*. And in this problem, the two events are *not* independent, because they both depend on the card that is the first down card in `P_first(n)` and becomes the up card in `P_all(n - 1)`. For example, suppose the first down card is 0, the lowest possible card. That certainly has a big effect on `P_first(n)`, because it means a guess of "higher" will always be wrong and a guess of "lower" will always be right. And it also has a big effect on guessing the rest of the sequence, because it means that the second guess should always be "higher" (than 0), which is guaranteed to be right. 

My suspicions were well-founded; I'll need something a bit more complicated.

# Second Try at Extra Credit

I need to break down the problem in a way that shares the information about the first down card (that becomes the next up card). One way to do that is to define `P_given(up, n)` to be the probability of guessing all the cards right *given* that the flipped-up card has index `up` in the sorted list of `n` remaining cards (including the `up` card).

I will still define `P_all(n)`, but it won't call itself recursively, rather it calls `P_given(up, n)` for every possible `up` card, and averages the results:

In [7]:
def P_all(n) -> float:
    """Probability of guessing all n cards right."""
    return mean(P_given(up, n) for up in range(n))

`P_given(up, n)` works by considering each possible first down card, computing if we would guess the first down card right given the up card  (by calling `first_right(up, down, n)`), and if right, calling `P_given` recursively with the old first down card (renumbered) being the new up card, and the deck size, `n`, being one card smaller. 
When this is all done, we take the average probability value over all the down cards. We also arrange for `P_given` to cache its values so that we don't repeat computations; this is crucial for efficiency.

In [8]:
@lru_cache(None)
def P_given(up, n) -> float:
    """Probability of guessing all n cards right, given the up card."""
    return (1 if n <= 2 else
            mean(first_right(up, down, n) 
                 and P_given(renumber(down, up), n - 1)
                 for down in range(n) if down != up))

`first_right(up, down, n)` returns true if we would correctly guess the down card given the up card. The optimal policy is to guess that the down card will be higher than the up card when the up card is in the lower half of `range(n)`. 

And finally, `renumber` renumbers a down card by decrementing it by one if the up card is lower (meaning that removing the up card from the deck makes the down card one closer to the lowest possible card). 

In [9]:
def first_right(up, down, n) -> bool:
    """Do we guess right given these up and down cards, 
    when the set of cards (including up and down) is range(n)?"""
    return (up > down) == (up > (n - 1) / 2)

def renumber(down, up) -> int:  return down - (up < down)

We can see that `P_all(n)` differs from the old brute-force result  only in the 17th decimal place (due to round-off error):

In [10]:
P_all(9)

0.17085537918871255

In [11]:
mean(map(guess_all, permutations(cards)))

0.17085537918871252

We can now efficiently answer the extra credit questions:

In [12]:
P_all(19)

0.008419397002884993

In [13]:
P_all(99)

6.67213407124781e-14

The moral is: don't expect to ever guess 99 cards all right, and allocate at least about 100 tries to guess 19 cards right.

# Computational Complexity

My first approach, with `guess_all`, is $O(n!)$ because it considers every permutation of $n$ cards.

For my second approach, `first_right` and `renumber` do only a constant amount of work each call. `P_all` makes $n$ calls to `P_given`, and `P_given` considers $n$ down cards and makes  recursive calls only when `first_guess(up, down, n)` returns true. It seems difficult to analyze how often that happens. But we can put an upper bound on the work `P_given` does. It is memoized (with `lru_cache`) and both of its arguments are restricted to $n$ possible values, so we can say that `P_given(up, n)` is $O(n^2)$ as an upper bound, and therefore `P_all(n)` is $O(n^3)$ as an upper bound.

However, experimental evidence suggests a stricter bound of $O(n^2)$ for `P_all(n)` (although I haven't proved it).

In the table below, `n` is the number of cards,  `s` is the size of the cache for `P_given` (that is, the number of computations `P_given` performed and cached during the computation of `P_all(n)`), and we see that, for all values of $n$ tested, `s` is  *exactly* equal to $c = n \times (n + 1) / 2 - 1$. 

In [14]:
print('|   n |      s | s==c | P_all(n) |')
print('|—————|————————|——————|——————————|')
for n in list(range(3, 10)) + list(range(10, 151, 10)):
    P_given.cache_clear()
    p = P_all(n)
    s = P_given.cache_info().currsize
    c = n * (n + 1) / 2 - 1
    print(f'|{n:4d} | {s:6,d} | {s == c} | {p:.2e} |')

|   n |      s | s==c | P_all(n) |
|—————|————————|——————|——————————|
|   3 |      5 | True | 8.33e-01 |
|   4 |      9 | True | 6.67e-01 |
|   5 |     14 | True | 5.17e-01 |
|   6 |     20 | True | 3.97e-01 |
|   7 |     27 | True | 3.01e-01 |
|   8 |     35 | True | 2.28e-01 |
|   9 |     44 | True | 1.71e-01 |
|  10 |     54 | True | 1.28e-01 |
|  20 |    209 | True | 6.18e-03 |
|  30 |    464 | True | 2.71e-04 |
|  40 |    819 | True | 1.14e-05 |
|  50 |  1,274 | True | 4.71e-07 |
|  60 |  1,829 | True | 1.91e-08 |
|  70 |  2,484 | True | 7.69e-10 |
|  80 |  3,239 | True | 3.07e-11 |
|  90 |  4,094 | True | 1.22e-12 |
| 100 |  5,049 | True | 4.83e-14 |
| 110 |  6,104 | True | 1.90e-15 |
| 120 |  7,259 | True | 7.49e-17 |
| 130 |  8,514 | True | 2.94e-18 |
| 140 |  9,869 | True | 1.15e-19 |
| 150 | 11,324 | True | 4.49e-21 |
